h1. Load the Outage Database (OD.csv). In the “FailureMode” field,   
h2. fix the inconsistent use of CAPitalisation.   
h3. Remove all numbers from the field.  


In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')

import pandas as pd
import numpy as np
import timeit 
from datetime import datetime
from datetime import timedelta
#NB : timeit.default_timer() auto chooses timeing function for os, nicer than time.time() or time.clock()
print ("pd.__version__:", pd.__version__)
print ("np.__version__:", np.__version__)

print ("libraries imported")

pd.__version__: 0.20.1
np.__version__: 1.12.1
libraries imported


In [2]:
start_time = timeit.default_timer()
print ("start loading data")
df1 = pd.read_csv('OD.csv')
elapsed = timeit.default_timer() - start_time
print ("elapsed time : ", elapsed)

start loading data
elapsed time :  0.004976675593831504


In [3]:
print (df1.shape)

(1000, 4)


In [4]:
df1.head(5)

,Start,Finish,AACodeDesc,FailureMode
0,29/07/2015 23:30,30/07/2015 3:34,Operational Stops,Pre-Drying tower low flow
1,20/03/2015 13:52,20/03/2015 13:55,Unscheduled Maintenance,Feed pots
2,7/10/2013 22:14,7/10/2013 22:27,Operational Stops,ESD LP cooling system temperatures / flows
3,28/01/2013 9:28,28/01/2013 10:15,Delays,Metallurgist testwork
4,14/04/2016 12:15,14/04/2016 12:21,Operational Stops,ESD LP cooling system temperatures / flows


In [5]:
#h2. fix the inconsistent use of CAPitalisation. 
df1['FailureMode'] = df1['FailureMode'].str.lower()

In [6]:
#h3. Remove all numbers from the field.
df1['FailureMode'] = df1['FailureMode'].str.replace('\d+', '')

i1. Which 3 areas (“FailureMode”) have the largest total time to repair? 
nb: group by area, calc total time, pick largest.

i2. Which 3 areas have the smallest median time between failures?
nb: group by area, calculate median for area, sort by median.

In [7]:
df1.dtypes

Start          object
Finish         object
AACodeDesc     object
FailureMode    object
dtype: object

In [8]:
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
print ("is_string:", is_string_dtype(df1['Start']))
print ("is_numeric:", is_numeric_dtype(df1['Start']))
print ("is_string:", is_string_dtype(df1['Finish']))
print ("is_numeric:", is_numeric_dtype(df1['Finish']))
type(df1['Finish'][0])

is_string: True
is_numeric: False
is_string: True
is_numeric: False


str

In [9]:
start_time = timeit.default_timer()
#suppress warning
#refer https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas
#https://pandas.pydata.org/pandas-docs/stable/options.html
#pd.options.mode.chained_assignment = None or 'warn'
pd.options.mode.chained_assignment = None
print ("start")
df1["duration"] = np.nan
for i in range(df1.shape[0]):
    tempFinish = datetime.strptime(df1['Finish'][i], '%d/%m/%Y %H:%M')
    #print ("type(tempFinish):", type(tempFinish))
    tempStart = datetime.strptime(df1['Start'][i], '%d/%m/%Y %H:%M')
    #print ("type(tempStart):", type(tempStart))
    df1['duration'][i] = (tempFinish - tempStart).total_seconds()
    #print (type(df1['duration'][i]))
    
    #df1["durationSeconds"][i] = (df1['duration'][i]-datetime(1970,1,1)).total_seconds()
elapsed = timeit.default_timer() - start_time
print ("elapsed time : ", elapsed)

start
elapsed time :  0.16260750523668654


In [10]:
df1['duration'].dtypes

dtype('float64')

In [11]:
#tempDuration = list(df1['duration'])
df1.loc[df1['duration'].idxmax()]


Start                2/02/2014 22:54
Finish               18/02/2014 6:00
AACodeDesc     Scheduled Maintenance
FailureMode         planned shutdown
duration                 1.32156e+06
Name: 57, dtype: object

In [12]:
rowMax = df1['duration'].idxmax()

In [13]:
df1.iloc[[rowMax]]

,Start,Finish,AACodeDesc,FailureMode,duration
57,2/02/2014 22:54,18/02/2014 6:00,Scheduled Maintenance,planned shutdown,1321560.0


In [14]:
#i1. Which 3 areas (“FailureMode”) have the largest total time to repair?   
#nb: group by area, calc total time, pick largest.

In [15]:
#https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html
aggFailureModeDuration = df1.groupby(['FailureMode'])['duration'].agg(['sum','count', 'median'])
#print (type(aggFailureModeDuration))
#print (aggFailureModeDuration.head())
print ("-----------------largest total 'FailureMode' area time to repair -----------------")
print (aggFailureModeDuration.sort_values('sum', ascending=False).head(10))
print ("-----------------smallest median 'FailureMode' area time to repair -----------------")
print (aggFailureModeDuration.sort_values('median', ascending=True).head(10) )

-----------------largest total 'FailureMode' area time to repair -----------------
                                                          sum  count    median
FailureMode                                                                   
planned shutdown                                    1664160.0      7   70740.0
stock accumulation                                   582120.0      2  291060.0
hp cooling system leaks                              520020.0     12   16170.0
clean uptake / accretion fall                        349860.0     24    8850.0
furnace change                                       286320.0     18   13560.0
slag reduction                                       263160.0      9    2700.0
acid plant delay                                     176460.0    118     660.0
esd hp cooling system temperatures / flows / pr...   127440.0     80     690.0
feed pots                                            118800.0     83     300.0
slow tapping rate / feed connection             

j. Challenge question (optional):     
Sensor “master_feed_rate” represents the master feed rate into the process.   
How confident are you that the timestamps in the Outage Database match the Process Database?    
